<a href="https://colab.research.google.com/github/netjerikhet/ai-human-emotions/blob/main/Photo_Animation_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone -q https://github.com/ChanduS06/dataset-generic Emotions-dir
!gdown --id 1YgqCAPEcrX9fvvqcPlO4b6WecTqZW_MA ## drive containing Nikita provided videos
!unzip archive.zip
!mkdir /content/Archive  # to store iterated videos in a folder

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1YgqCAPEcrX9fvvqcPlO4b6WecTqZW_MA
To: /content/archive.zip
100% 772M/772M [00:07<00:00, 100MB/s]
Archive:  archive.zip
  inflating: stor/n.koritsky/data/deeperforensics/source_videos/W026/light_uniform/disgust/camera_front/W026_light_uniform_disgust_camera_front.mp4  
  inflating: stor/n.koritsky/data/deeperforensics/source_videos/W026/light_uniform/sad/camera_front/W026_light_uniform_sad_camera_front.mp4  
  inflating: stor/n.koritsky/data/deeperforensics/source_videos/W026/light_uniform/angry/camera_front/W026_light_uniform_angry_camera_front.mp4  
  inflating: stor/n.koritsky/data/deeperforensics/source_videos/W026/light_uniform/neutral/camera_front/W026_light_uniform_neutral_camera_front.mp4  
  inflating: stor

In [2]:
# to iterate and store all videos from subdirectory under unified directory "Archive"

import os
import os.path
import shutil

folder = r"stor"
destinationpath = '/content/Archive'
for dirpath, dirnames, filenames in os.walk("stor"):
    for filename in [f for f in filenames if f.endswith(".mp4")]:
        shutil.move(os.path.join(dirpath, filename), os.path.join(destinationpath, filename))

In [3]:
# Extraction of labelled emotion and using them to associate it to the file

import csv
from os import listdir
from os.path import isfile, join
import re
from itertools import zip_longest

onlyfiles = [f for f in listdir('/content/Archive') if isfile(join('/content/Archive', f))]
emotions_cat = []
for f in onlyfiles:
  emotions_cat.extend(re.findall(r"surprise|disgust|angry|fear|happy|contempt|neutral|sad", f))

# csv-data incase if needed (optional)
export_data = zip_longest(*[onlyfiles,emotions_cat], fillvalue = '')  # or vice-versa
with open('result.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames = ["Files", "Emotions"])
    writer.writeheader()
    writer = csv.writer(f)
    for val in zip(onlyfiles,emotions_cat):
      # writer.writerow(("file", "emotion"))
      writer.writerows(export_data)

data = dict(zip(onlyfiles, emotions_cat)) # key-value pairs || Key = filename, value = Emotion

# data
# onlyfiles
# len(onlyfiles)

In [4]:
!mkdir /content/Archive_Images #(required)
# !rm -r /content/Archive_Images

In [5]:
!pip install deepface

     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 2.3 MB 11.1 MB/s 
     |████████████████████████████████| 462 kB 42.7 MB/s 


In [6]:
# Extract frames from all videos to to a folder (to determine Gender)
import cv2

def getFirstFrame(videofile, filename, flag=False):
    vidcap = cv2.VideoCapture(videofile)
    success, image = vidcap.read()
    if success:
      if flag:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,2000)      # cue to 4 sec. position
        success,image = vidcap.read()
        cv2.waitKey()
      cv2.imwrite("/content/Archive_Images/{}.jpg".format(filename), image)  # save frame as JPEG file

for filename in data.keys():
  if filename == 'W008_light_uniform_neutral_camera_front.mp4' or \
  filename == 'W031_light_uniform_surprise_camera_front.mp4' or \
  filename == 'W042_light_uniform_disgust_camera_front.mp4' or \
  filename == 'W042_light_uniform_fear_camera_front.mp4' or \
  filename == 'W007_light_uniform_neutral_camera_front.mp4' or \
  filename == 'W024_light_uniform_surprise_camera_front.mp4' or \
  filename == 'W042_light_uniform_contempt_camera_front.mp4' or \
  filename == 'W024_light_uniform_happy_camera_front.mp4' or \
  filename == 'W042_light_uniform_happy_camera_front.mp4' or \
  filename == 'W042_light_uniform_angry_camera_front.mp4':
    getFirstFrame(videofile='/content/Archive/{}'.format(filename), 
                  filename=filename, flag=True)
  else:
    getFirstFrame(videofile='/content/Archive/{}'.format(filename), 
                  filename=filename)

In [7]:
# determine Gender. 
# Format desired: {W007_abc.mp4: {"sad", "male"}}
# Why filename as key? Dict needs unique keys(-value) pairs

from deepface import DeepFace
new_data = {}
for filename, emotion in data.items():
  if filename == 'W036_light_uniform_surprise_camera_front.mp4' or filename == 'W042_light_uniform_angry_camera_front.mp4':
    continue
  obj = DeepFace.analyze(img_path = "/content/Archive_Images/{}.jpg".format(filename), actions = ['gender', 'race'])
  new_data[filename] = [emotion, obj['gender'], obj['race']]
new_data

Directory  /root /.deepface created
Directory  /root /.deepface/weights created
gender_model_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/gender_model_weights.h5
To: /root/.deepface/weights/gender_model_weights.h5
100%|██████████| 537M/537M [00:06<00:00, 80.8MB/s]


race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/race_model_single_batch.h5
To: /root/.deepface/weights/race_model_single_batch.h5
100%|██████████| 537M/537M [00:10<00:00, 51.9MB/s]
Action: race: 100%|██████████| 2/2 [00:00<00:00,  2.36it/s]


{'W005_light_uniform_angry_camera_front.mp4': ['angry',
  'Woman',
  {'asian': 0.07712738925805858,
   'black': 0.022877733680279844,
   'indian': 0.8322604061377533,
   'latino hispanic': 15.053884989486335,
   'middle eastern': 28.721231665758093,
   'white': 55.29262486608325}],
 'W005_light_uniform_contempt_camera_front.mp4': ['contempt',
  'Woman',
  {'asian': 0.168041976405947,
   'black': 0.0018701276736107387,
   'indian': 0.03960725971019326,
   'latino hispanic': 2.8063934106465225,
   'middle eastern': 2.832605113542385,
   'white': 94.15147865721457}],
 'W005_light_uniform_disgust_camera_front.mp4': ['disgust',
  'Woman',
  {'asian': 4.147790744900703,
   'black': 0.23520002141594887,
   'indian': 2.7348294854164124,
   'latino hispanic': 13.923272490501404,
   'middle eastern': 18.084703385829926,
   'white': 60.87420582771301}],
 'W005_light_uniform_fear_camera_front.mp4': ['fear',
  'Woman',
  {'asian': 0.04425821825861931,
   'black': 0.007432114216499031,
   'indian': 

In [28]:
# !mkdir -p data/vox-png ## required as vox-256.yaml contains data/vox-png as root_dir

In [ ]:
# !CUDA_VISIBLE_DEVICES=0 python run.py --config config/vox-256.yaml --mode reconstruction --checkpoint vox-cpk.pth.tar

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Use random train-test split.
Traceback (most recent call last):
  File "run.py", line 72, in <module>
    dataset = FramesDataset(is_train=(opt.mode == 'train'), **config['dataset_params'])
  File "/content/frames_dataset.py", line 83, in __init__
    train_videos, test_videos = train_test_split(self.videos, random_state=random_seed, test_size=0.2)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2421, in train_test_split
    n_samples, test_size, train_size, default_test_size=0.25
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py", line 2101, in _validate_shuffle_split
    "aforementioned parameters.".format(n

In [8]:
!pip install ffmpy &> /dev/null
!git init -q .
!git remote add origin https://github.com/AliaksandrSiarohin/first-order-model
!git pull -q origin master
!git clone -q https://github.com/graphemecluster/first-order-model-demo demo 
!pip install imageio matplotlib numpy pandas python-dateutil pytz PyYAML scikit-image scikit-learn scipy torch torchvision tqdm

In [9]:
import os, requests, numpy
import imageio
import IPython.display
import PIL.Image
import imageio
import io
import ipywidgets
import os.path
import requests
import skimage.transform
import warnings
from demo import load_checkpoints, make_animation
from ffmpy import FFmpeg
from skimage import img_as_ubyte
import skimage.transform

In [15]:
a=b=c=d=e=f=g=h=i=j=k=0
for file,data in new_data.items():
  if data[1]=='Woman':
    a+=1
  # if data[1]=='Man':
  #   b+=1 
    if data[0]=='sad':
      c+=1
    if data[0]=='surprise':
      d+=1
    if data[0]=='neutral':
      e+=1
    if data[0]=='happy':
      f+=1
    if data[0]=='fear':
      g+=1
    if data[0]=='disgust':
      h+=1
    if data[0]=='contempt':
      i+=1
    if data[0]=='angry':
      j+=1       
  k+=1 
print(a,b,c,d,e,f,g,h,i,j,k)  

90 0 13 13 8 12 11 10 13 10 246


In [10]:
# method to feed/resize image with aspired dimensions
def resize(image, size=(256, 256)):
    w, h = image.size
    d = min(w, h)
    r = ((w - d) // 2, (h - d) // 2, (w + d) // 2, (h + d) // 2)
    return image.resize(size, resample=PIL.Image.LANCZOS, box=r)

In [11]:
# input image manually/locally
from google.colab import files
from PIL import Image
image = files.upload()
img_name = [*image]
selected_image = resize(PIL.Image.open('{}'.format(img_name[0])).convert("RGB"))

Saving 34.png to 34.png


In [ ]:
# input image through web links (input URL)

import os
upload = files.upload()
img_data = requests.get("https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSa1INgadp1f7fPVyip1SJ4Z4nJRO6L6ZarPRK1UeZEhPHnegTEK8-v6st82lDk1XO5D3M&usqp=CAU").content
with open('einstein.png', 'wb') as handler:
    handler.write(img_data)

selected_image = resize(PIL.Image.open('einstein.png').convert("RGB")) 

In [ ]:
# For Deepface verification method only
# from deepface import DeepFace

# metrics = ["cosine", "euclidean", "euclidean_l2"]

# # for Face-alignment verification only
# import face_alignment

# # sfd for SFD, dlib for Dlib and folder for existing bounding boxes.
# fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, face_detector='sfd')

In [12]:
# Models/checkpoints, if required can be changed here
# Pre-trained checkpoint chosen:  
# VoxCeleb is an audio-visual dataset consisting of short clips of human speech, extracted from interview videos uploaded to YouTube
# https://github.com/AliaksandrSiarohin/video-preprocessing: Preprocessing of voxceleb dataset, in detail

os.makedirs("user", exist_ok=True)
checkpoint = 'vox-cpk.pth.tar' #model.value + ('' if model.value == 'fashion' else '-cpk') + '.pth.tar'
download = requests.get(requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key=https://yadi.sk/d/lEw8uRm140L_eQ&path=/' + checkpoint).json().get('href'))
if not os.path.isfile(checkpoint):
		with open(checkpoint, 'wb') as f:
			f.write(download.content)
	 
#alternate way to download vox checkpoint (TO-DO: option to choose from various checkpoints)
# !gdown --id 1_v_xW1V52gZCZnXgh1Ap_gwA9YVIzUnS ## drive containing vox-cpk.pth.tar
# !gzip vox-cpk.pth.tar

In [34]:
def perform_animation(checkpoint, selected_image, new_data, user_input):
	generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path=checkpoint)
	for file, emotion_gender in new_data.items():
		if file == "W019_light_uniform_surprise_camera_front.mp4":
		# if emotion_gender[1] == user_input[1]:  # 0=> emotion, 1=> Gender
		# 	if emotion_gender[0] == user_input[0]: 
		# if race_input == user_input[2]:
			print("predicting animation for gender: {}, emotion: {} for file: {}".format(emotion_gender[1], emotion_gender[0], file))
			driving_video = []
			reader = imageio.get_reader('/content/Archive/{}'.format(file), mode='I', format='FFMPEG')
			fps = reader.get_meta_data()['fps']
			for frame in reader:
				driving_video.append(frame)
			imageio.mimsave('/content/resultant_videos/{}__driving.mp4'.format(file), [skimage.transform.resize(frame, (256, 256)) for frame in driving_video], fps=fps)
			predictions = make_animation(
				skimage.transform.resize(numpy.asarray(selected_image), (256, 256)),
				[skimage.transform.resize(frame, (256, 256)) for frame in driving_video],
				generator,
				kp_detector,
				relative=True,
				adapt_movement_scale=True
			)
			imageio.mimsave('/content/resultant_videos/{}.mp4'.format(file), [img_as_ubyte(frame) for frame in predictions], fps=fps)
		continue
	# FFmpeg(inputs={'temp.mp4': None, selected_video: None}, outputs={'output.mp4': '-c copy -y'}).run()

In [14]:
!mkdir /content/resultant_videos

In [35]:
#Logic:
emotion_input = input("Input Emotion. Available optiona Male, Woman  ")
# race_input = input("Input race. Available options: ")

perform_animation(checkpoint=checkpoint, 
                  selected_image=selected_image,
                  new_data=new_data,
                  user_input=[emotion_input, gender_input])

Input Emotion. Available optiona Male, Woman  a
predicting animation for gender: Man, emotion: surprise for file: W019_light_uniform_surprise_camera_front.mp4


100%|██████████| 200/200 [00:30<00:00,  6.49it/s]


In [ ]:
import face_alignment
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
cap = cv2.VideoCapture('output.mp4')
frames = []
while True:
    success, frame = cap.read()
    if not success:
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frames.append(frame)

In [ ]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, device='cpu', face_detector='blazeface')

Downloading: "https://github.com/hollance/BlazeFace-PyTorch/blob/master/blazeface.pth?raw=true" to /root/.cache/torch/hub/checkpoints/blazeface.pth


  0%|          | 0.00/410k [00:00<?, ?B/s]

Downloading: "https://github.com/hollance/BlazeFace-PyTorch/blob/master/anchors.npy?raw=true" to /root/.cache/torch/hub/checkpoints/anchors.npy


  0%|          | 0.00/28.1k [00:00<?, ?B/s]

In [ ]:
import time
t_start = time.time()
preds = fa.get_landmarks_from_image(frames[0])
print(f'BlazeFace: Execution time for a single image: {time.time() - t_start}')

BlazeFace: Execution time for a single image: 6.867302417755127


In [ ]:
plt.imshow(frames[0])
for detection in preds:
    plt.scatter(detection[:,0], detection[:,1], 2)

In [ ]:
batch = np.stack(frames)
batch = batch.transpose(0, 3, 1, 2)
batch = torch.Tensor(batch[:2])
t_start = time.time()
preds = fa.get_landmarks_from_batch(batch)
print(f'BlazeFace: Execution time for a batch of 2 images: {time.time() - t_start}')

BlazeFace: Execution time for a batch of 2 images: 1.9059646129608154


In [ ]:
fig = plt.figure(figsize=(10, 25))

for i, pred in enumerate(preds):
    plt.subplot(5, 2, i + 1)
    plt.imshow(frames[i])
    plt.title(f'frame[{i}]')
    for detection in pred:
        plt.scatter(detection[:,0], detection[:,1], 2)

IndexError: ignored

In [ ]:
!git clone --recursive https://github.com/AliaksandrSiarohin/pose-evaluation

Cloning into 'pose-evaluation'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 55 (delta 17), reused 9 (delta 9), pack-reused 32
Unpacking objects: 100% (55/55), done.
Submodule 'OpenFacePytorch' (https://github.com/thnkim/OpenFacePytorch) registered for path 'OpenFacePytorch'
Submodule 'face-alignment' (https://github.com/1adrianb/face-alignment) registered for path 'face-alignment'
Submodule 'pose_estimation' (https://github.com/AliaksandrSiarohin/pytorch_Realtime_Multi-Person_Pose_Estimation) registered for path 'pose_estimation'
Submodule 'reid_baseline' (https://github.com/layumi/Person_reID_baseline_pytorch) registered for path 'reid_baseline'
Cloning into '/content/pose-evaluation/OpenFacePytorch'...
remote: Enumerating objects: 56, done.        
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56        
Cloning into '/content/pose-evaluation/face-alignment'...
remo

In [ ]:
!git clone https://github.com/1adrianb/face-alignment face-alignment

Cloning into 'face-alignment'...
remote: Enumerating objects: 1026, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1026 (delta 24), reused 34 (delta 13), pack-reused 969
Receiving objects: 100% (1026/1026), 5.26 MiB | 15.22 MiB/s, done.
Resolving deltas: 100% (625/625), done.


In [ ]:
cd pose-evaluation/face-alignment

/content/pose-evaluation/face-alignment


In [ ]:
!pip install opencv-python scipy>=0.17.0 scikit-image
!python setup.py install

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_info
creating face_alignment.egg-info
writing face_alignment.egg-info/PKG-INFO
writing dependency_links to face_alignment.egg-info/dependency_links.txt
writing requirements to face_alignment.egg-info/requires.txt
writing top-level names to face_alignment.egg-info/top_level.txt
writing manifest file 'face_alignment.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'face_alignment.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/face_alignment
copying face_alignment/utils.py -> build/lib/face_alignment
copying face_alignment/api.py -> build/lib/face_alignment
c

In [ ]:
cd ..

/content/pose-evaluation


In [ ]:
# to resize images present in the directory, useful only when external files uploaded or downloaded 
# image gives the error array size error (256,256,3) => 1,96,608
from PIL import Image

basewidth = 256
baseheight = 256
img = Image.open('/content/Emotions-dir/Images/einstein.png')
wpercent = (basewidth / float(img.size[0]))
hsize = int((float(img.size[1]) * float(wpercent)))
img = img.resize((basewidth, hsize), Image.ANTIALIAS)
hpercent = (baseheight / float(img.size[1]))
wsize = int((float(img.size[0]) * float(hpercent)))
img = img.resize((wsize, baseheight), Image.ANTIALIAS)
img.save('/content/Emotions-dir/Images/resized_image.png')

In [ ]:
#  For extracting face keypoints (AKD)

#  Evaluation is based on several pretrained models for faces and human poses. We evaluate, 
# if the movement is the same as in target video and if the identity is preserved.

#  The general pipeline first compute the statistics for real data and generated using 
# the script extract.py. Then use script cmp.py to measure the final score. (from pose-evaluation clone)

!python extract.py --in_folder /content/Emotions-dir/Images --out_file pose_gt.pkl --is_video --type face_pose --image_shape 256,256
!python extract.py --in_folder /content/Emotions-dir/videos --out_file pose_gen.pkl --is_video --type face_pose --image_shape 256,256
!python cmp_kp.py pose_gt.pkl pose_gen.pkl

 50% 1/2 [00:03<00:03,  3.23s/it]
Traceback (most recent call last):
  File "extract.py", line 204, in <module>
    df = func(**vars(args))
  File "extract.py", line 57, in extract_face_pose
    video = frames2array(os.path.join(in_folder, file), is_video, image_shape, column)
  File "/content/pose-evaluation/util.py", line 37, in frames2array
    return video[:, :, (image_shape[1] * column):(image_shape[1] * (column + 1))]
IndexError: too many indices for array
100% 1/1 [00:29<00:00, 30.00s/it]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/pickle.py", line 166, in try_read
    return read_wrapper(lambda f: pkl.load(f))
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/pickle.py", line 147, in read_wrapper
    is_text=False)
  File "/usr/local/lib/python3.7/dist-packages/pandas/io/common.py", line 406, in _get_handle
    f = open(path_or_buf, mode)
FileNotFoundError: [Errno 2] No such file or directory: 'pose_gt.pkl'

During handling